<a href="https://colab.research.google.com/github/raqcrisb/Project_Aula05_raqcrisb/blob/main/Projeto_Seu_Tipo_Ideal_de_Bike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar o cliente para poder se comunicar com o genai

In [3]:
!pip install -q -U google-generativeai

Fazer a importação desse SDK

In [4]:
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab  import userdata
api_key=userdata.get('secret_Key')
genai.configure(api_key=api_key)

Utilizando o modelo embedContent

In [110]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [111]:
title="Seu tipo ideal de bike"
sample_text = ("Título: Com base em seu perfil iremos indicar o tipo de bike que melhor combina com o seu estilo."
"\n"
"Descreva em poucas palavras como você é. Por exemplo, sou um aventureiro!"
"\n")
embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(sample_text)
print(embeddings)


Título: Com base em seu perfil iremos indicar o tipo de bike que melhor combina com o seu estilo.
Descreva em poucas palavras como você é. Por exemplo, sou um aventureiro!

{'embedding': [-0.04639202, -0.037180174, -0.018308146, 0.016574988, 0.06139608, -0.012829435, -0.009676454, 0.063850716, -0.0073123085, 0.01364082, -0.0034763494, 0.044964537, -0.01765349, -0.052921887, 0.016672652, -0.018587586, -0.037301663, 0.001071139, -0.0032537123, 0.005254738, -0.013426251, -0.033852756, -0.008693527, -0.009617694, -0.0053453483, -0.031998068, 0.017541416, -0.008199677, -0.007144935, 0.043903574, -0.03994702, 0.05854197, -0.035064463, -0.010897379, 0.009906689, -0.045485947, -0.04472963, 0.02488523, 0.045378186, -0.0031500831, -0.00806075, -0.028823817, -0.04052342, -0.024878465, 0.007821343, 0.004538123, 0.008175279, 0.018809875, -0.00014889883, -0.032643743, -0.046931297, -0.011199694, 0.07912625, -0.009209581, -0.025772398, -0.017092148, 0.024875013, -0.019595258, -0.03578614, 0.002901753

Listagem de documentos que serão buscados

In [112]:
DOCUMENT1 = {
   "Título":"Aventureiro",
   "Conteudo":"O modelo montain bike é o ideal para quem gosta da natureza, como pedalar por horas em montanhas ou campos abertos. Também conhecido como modelos MTB, estas bicicletas são mais robustas e mais resistentes. Possuem marchas que ajudam a enfrentar uma subida forte e contam com suspensão (dianteira e/ou traseira) possibilitando conforto e estabilidade em locais irregulares. É o modelo ideal para uma pessoa de gosta de aventura e agitação."
}
DOCUMENT2 = {
   "Título":"Acelerado",
   "Conteudo":"O modelo speed é pra quem gosta de vento na cara, velocidade, pista e uma boa performance. Por isso, este modelo também é conhecido como também conhecido como ciclismo de estrada. Estas bicicletas são leves e posuem uma boa aerodinâmica com segurança e conforto. Foram elaboradas para maximizar a eficiência e otimizar o desempenho. Estas pessoas são mais esportitas e gostam de pedalar em bando"
}
DOCUMENT3 = {
   "Título":"Tranquilo",
   "Conteudo":"O modelo urbano é para uma pessoa tranquila. Ele tem como característica principal a locomoção mais correlacionada com o laser, mas que temabém pode ser utilizada como meio de transporte para o trabalho. Este modelo pode contar com bagageiro e opcionais para evitar que a pessoa se suje, como protetor de corrente e para lamas. Também é indicado para pessoas sonhadoras e que gostam de estar com a família ou com alguns poucos amigos."
}
DOCUMENT4 = {
   "Título":"Acomodado",
   "Conteudo":"O modelo elétrico pode conter pedalada assistida na qual o motor amplifica a pedalada do ciclista ou pode ser aquele modelo que o ciclista nem precisa pedalar. Ela funciona com uma bateria recarregável e é indicada no percurso de longos trechos evitando assim que o ciclista chegue suado e cansado no seu destino. É o modelo ideal para uma pessoa mais acomodada ou que não deseja se esforçar muito."
}


documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3, DOCUMENT4]

In [113]:
# Passar para o formato datasheet
df = pd.DataFrame(documents)

In [114]:
df.columns = ["Título", "Conteúdo"]
df

,Título,Conteúdo
0,Aventureiro,O modelo montain bike é o ideal para quem gost...
1,Acelerado,O modelo speed é pra quem gosta de vento na ca...
2,Tranquilo,O modelo urbano é para uma pessoa tranquila. E...
3,Acomodado,O modelo elétrico pode conter pedalada assisti...


In [115]:
model = "models/embedding-001"

Método para trabalhar com o datasheet.

O resultado vai ser inserido na nova coluna

In [116]:
def embed_fn(title, texto):
  return genai.embed_content(model=model,
                                 content=texto,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

Criar uma nova coluna

In [117]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Título"], row["Conteúdo"]), axis=1)
df

,Título,Conteúdo,Embeddings
0,Aventureiro,O modelo montain bike é o ideal para quem gost...,"[-0.059365977, -0.067413956, -0.013067978, 0.0..."
1,Acelerado,O modelo speed é pra quem gosta de vento na ca...,"[0.016806815, -0.038365476, -0.01666913, 0.037..."
2,Tranquilo,O modelo urbano é para uma pessoa tranquila. E...,"[-0.0002885401, -0.049983893, -0.03719459, 0.0..."
3,Acomodado,O modelo elétrico pode conter pedalada assisti...,"[-0.02872042, -0.02650506, -0.058923826, -0.00..."


Procura dentro desta lista de produtos escalares qual que tem a maior simularidade e obtém o ID dele.

In [118]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(df["Embeddings"]),embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteúdo"]

In [119]:
generation_config_lista = {
    "candidate_count": 1,
    "temperature": 0.5,
}

In [120]:
# Pergunta e resposta
consulta = "sou uma pessoa positiva, qual bike é melhor pra mim ?"
trecho = gerar_e_buscar_consulta(consulta, df, model)
# print(trecho)

In [122]:
prompt = f"Mude o texto a cada execução, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config_lista)
response = model_2.generate_content(prompt)

print(sample_text)
print(response.text)

Título: Com base em seu perfil iremos indicar o tipo de bike que melhor combina com o seu estilo.
Descreva em poucas palavras como você é. Por exemplo, sou um aventureiro!

O modelo elétrico pode ser equipado com pedalada assistida, onde o motor auxilia a pedalada do ciclista, ou pode ser um modelo sem necessidade de pedalar. Alimentado por uma bateria recarregável, ele é adequado para longos percursos, evitando que o ciclista chegue exausto ao seu destino. É uma opção ideal para aqueles que buscam conforto ou desejam minimizar o esforço físico.
